In [135]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras import layers, models
from urllib.parse import urlparse
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [137]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [139]:
#loading data
df = pd.read_csv("hf://datasets/zefang-liu/phishing-email-dataset/Phishing_Email.csv")

In [140]:
df.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [141]:
df.columns

Index(['Unnamed: 0', 'Email Text', 'Email Type'], dtype='object')

In [142]:
df['Email Type'].nunique()

2

In [143]:
df.isnull().sum()

Unnamed: 0     0
Email Text    16
Email Type     0
dtype: int64

In [144]:
df=df.dropna()

In [145]:
#convertin the email contents to lower case
df['Email Text'] = df['Email Text'].str.lower()

In [146]:
#reomving special characters
df['Email Text'] = df['Email Text'].apply(lambda x: re.sub(r'\W', ' ', x))

In [147]:
# removing extra spaces
df['Email Text'] = df['Email Text'].str.replace(r'\s+', ' ', regex=True).str.strip() 

In [148]:
# Label encoding
df['Email Type'] = df['Email Type'].map({'Safe Email': 0, 'Phishing Email': 1})

In [149]:
# Tokenization and stop word removal
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize_and_remove_stopwords(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    filtered_lemmas = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(filtered_lemmas)

# Apply the function to the email column
df['Email Text'] = df['Email Text'].apply(tokenize_and_remove_stopwords)

# Display the updated DataFrame
print(df['Email Text'])

0        6 1100 disc uniformitarianism 1086 sex lang di...
1        side galicismos galicismo spanish term name im...
2        equistar deal ticket still available assist ro...
3        hello hot lil horny toy one dream open minded ...
4        software incredibly low price 86 lower drapery...
                               ...                        
18645    date lonely housewife always wanted date lonel...
18646    request submitted access request anita dupont ...
18647    important prc mtg hi dorn john discovered rece...
18648    press clipping letter californian utility plea...
18649                                                empty
Name: Email Text, Length: 18634, dtype: object


In [161]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df['Email Text'], df['Email Type'], test_size=0.2, random_state=42)

In [163]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [173]:
# Build model
model = models.Sequential()
model.add(layers.Input(shape=(X_train_tfidf.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [177]:
# Train model
history = model.fit(X_train_tfidf, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 87s 232ms/step - accuracy: 0.9287 - loss: 0.3553 - val_accuracy: 0.9738 - val_loss: 0.0793
Epoch 2/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 86s 230ms/step - accuracy: 0.9845 - loss: 0.0486 - val_accuracy: 0.9762 - val_loss: 0.0592
Epoch 3/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 86s 232ms/step - accuracy: 0.9883 - loss: 0.0292 - val_accuracy: 0.9769 - val_loss: 0.0538
Epoch 4/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 141s 229ms/step - accuracy: 0.9888 - loss: 0.0253 - val_accuracy: 0.9772 - val_loss: 0.0518
Epoch 5/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 87s 232ms/step - accuracy: 0.9850 - loss: 0.0243 - val_accuracy: 0.9772 - val_loss: 0.0534
Epoch 6/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 142s 231ms/step - accuracy: 0.9873 - loss: 0.0215 - val_accuracy: 0.9765 - val_loss: 0.0505
Epoch 7/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 142s 231ms/step - accuracy: 0.9878 - loss: 0.0202 - val_accuracy: 0.9789 - val_loss: 0.0498
Epoch 8/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 87s 232ms/step - accuracy: 0.9888 - loss

In [179]:
import matplotlib.pyplot as plt
# Plotting the training history
def plot_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='best')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')

    plt.tight_layout()
    plt.show()

plot_history(history)

In [181]:
# Evaluate model
loss, accuracy = model.evaluate(X_test_tfidf, y_test)

117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9752 - loss: 0.0580


In [185]:
# Save the model
model.save('phishing_email_detection.keras')